In [1]:
%reload_ext dotenv
%dotenv

In [2]:
!pip install -qU python-dotenv
!pip install -qU PyPDF2
!pip install -qU langchain
!pip install -qU langchain-community
!pip install -qU langchain-core
!pip install -qU langchain-text-splitters
!pip install -qU langchain_upstage
!pip install -qU oracledb
print("======== Job Completed ========")

======== Job Completed ========


In [4]:
import sys
import array
import time
import os
from dotenv import load_dotenv

import oracledb
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS

from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import BaseDocumentTransformer, Document

from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import warnings
warnings.filterwarnings("ignore")

print("Successfully imported libraries and modules")

Successfully imported libraries and modules


In [26]:
username=os.environ["DB_USER"]
password=os.environ["DB_PASSWORD"]
dsn=os.environ["DSN"]

con = oracledb.connect(user=username, password=password, dsn=dsn)

try: 
    conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!", conn23c.version)
except Exception as e:
    print("Connection failed!")

Connection successful! 23.4.1.24.6


In [27]:
from langchain_upstage import UpstageLayoutAnalysisLoader
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

def document(file_path, company):
    #file_path = "./oracle-database-23ai-new-features-guide.pdf"
    #file_path = "./info/4_위니아_2023en.pdf"
    layzer = UpstageLayoutAnalysisLoader(file_path, split="page")

    # For improved memory efficiency, consider using the lazy_load method to load documents page by page.
    docs = layzer.load()  # or layzer.lazy_load()

    text_splitter = RecursiveCharacterTextSplitter.from_language(
        chunk_size=1500, chunk_overlap=200, language=Language.HTML
    )
    docs = text_splitter.split_documents(docs)

    for doc in docs:
        doc.metadata['company'] = company

    return docs


docs1 = document("./info/1_LG_2022.pdf", "LG")
docs2 = document("./info/2_삼성전자_2023(ENG).pdf", "Samsung")
docs3 = document("./info/3_캐리어_2023.pdf", "Carrier")
docs4 = document("./info/4_위니아_2023en.pdf", "Winia")
docs5 = document("./info/5_미쓰비시일렉트릭_2023.pdf", "Mitsubishi")
docs6 = document("./info/6_파나소닉그룹_2023.pdf", "Panasonic")
docs7 = document("./info/7_한솔_2023.pdf", "Hansol")
docs8 = document("./info/8_하이메이드_2023.pdf", "Himade")
docs9 = document("./info/9_센추리_2023.pdf", "Century")
docs10 = document("./info/10_하이얼_2023.pdf", "Haier")

In [28]:
docs = docs1 + docs2 + docs3 + docs4 + docs5 + docs6 + docs7 + docs8 + docs9 + docs10

In [29]:
from langchain_upstage import UpstageEmbeddings
 
upstage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
     
# Configure the vector store with the model, table name, and using the indicated distance strategy for the similarity search and vectorize the chunks
s1time = time.time()

knowledge_base = OracleVS.from_documents(docs, upstage_embeddings, client=conn23c, 
                    table_name="text_embeddings2", 
                    distance_strategy=DistanceStrategy.DOT_PRODUCT)    

s2time =  time.time()      
print( f"Vectorizing and inserting chunks duration: {round(s2time - s1time, 1)} sec.")

Vectorizing and inserting chunks duration: 1530.3 sec.


In [15]:
vector_store = OracleVS(client=conn23c, 
                        embedding_function=upstage_embeddings,
                        table_name="text_embeddings2", 
                        distance_strategy=DistanceStrategy.DOT_PRODUCT)


companies = ["LG", "Samsung", "Carrier", "Winia", "Mitsubishi", "Panasonic", "Hansol", "Himade", "Century", "Haier"]


user_question = ("What is the investment plan that the company LG allocated to support its climate change transition plan?")
print ("The prompt to the LLM will be:",user_question)

for company in companies:
    if company in user_question[0]:
        retriever = vector_store.as_retriever(search_kwargs={'filter': {'company':company}})


if user_question:
    s3time =  time.time()
    result_chunks=knowledge_base.similarity_search(user_question)
    s4time = time.time()
    print(f"Search for the user question in the Oracle Database 23ai and return similar chunks duration: {round(s4time - s3time, 1)} sec.")

result_chunks


The prompt to the LLM will be: What specific steps is the company taking to reduce GHG emissions in line with the Paris Agreement?
Search for the user question in the Oracle Database 23ai and return similar chunks duration: 1.3 sec.


[Document(metadata={'page': 24}, page_content="<p id='57' data-category='paragraph' style='font-size:16px'>현대위아 환경경영 추진조직은 매월 기후변화 리스크를 식별 및 평가<br>하고 있으며, 기후변화 리스크에 대한 전사적인 실행 계획을 수립하<br>고 온실가스 감축 목표 및 관련 투자 의사결정을 지원하고 있습니다.<br>식별된 기후변화 리스크 중 중대한 이슈로 평가된 경우는 CEO 및 이<br>사회에 보고되어 최종적으로 의사결정이 이루어 집니다.</p> <p id='58' data-category='paragraph' style='font-size:16px'>또한 현대위아는 기후변화 관련 리스크를 구조화하여 관리하기 위해<br>환경정책 및 관리규정을 명문화하고 환경경영시스템을 구축・운영하<br>며, 환경경영 관련 법규를 지속적으로 모니터링합니다.</p> <br><p id='59' data-category='paragraph' style='font-size:16px'>이와 더불어 현대위아는 ESG 평가결과를 KPI별 성과에 반영하고,<br>유관 부서 팀장급 이상 관리자의 KPI에 온실가스 배출량 원단위 개선<br>율 등 환경관리 지표를 포함시킴으로써 기후변화 관련 리스크가 전사<br>리스크 관리체계에 통합될 수 있도록 체계를 구축하였습니다.</p> <h1 id='60' style='font-size:18px'>기후변화 리스크 관리 체계</h1> <p id='61' data-category='paragraph' style='font-size:18px'>2˚C 이하 시나리오 등 다양한 기후변화 관련 시나리오를 고려한 경영전략의 유연성</p> <br><p id='62' data-category='paragraph' style='font-size:16px'>현대위아는 다양한 기후변화 시나리오 상황에서 현대위아와 관련된 기후변화 리스크 및 기회가 미칠 수 있는 잠재적 재무영향에 대하여 

In [22]:
from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage, SystemMessage
 
llm = ChatUpstage()

template = """Answer the question based only on the following context:
              {context} 
              Question: {question} 
              """
prompt = PromptTemplate.from_template(template)


In [25]:
#user_question = ("What is the investment plan that the company allocated to support its climate change transition plan?")

s5time = time.time()
print("We are sending the prompt and RAG context to the LLM, wait a few seconds for the response...")
chain = (
  {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )
response = chain.invoke(user_question)
print("Question: ", user_question)
print("-"*100)
print(response)

s6time = time.time()
print("")
print( f"Send user question and ranked chunks to LLM and get answer duration: {round(s6time - s5time, 1)} sec.")

We are sending the prompt and RAG context to the LLM, wait a few seconds for the response...
Question:  What is the investment plan that the company allocated to support its climate change transition plan?
----------------------------------------------------------------------------------------------------
The investment plan that the company allocated to support its climate change transition plan is not explicitly mentioned in the provided context.

Send user question and ranked chunks to LLM and get answer duration: 2.7 sec.
